In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [3]:
!pwd

/home/marcnaweb/code/marcnaweb/car_recommendation_engine/notebooks


In [4]:
price_df = pd.read_csv('/home/marcnaweb/code/marcnaweb/car_recommendation_engine/raw_data/car_prices_w_prices_scaled.csv', index_col=0)

In [5]:
price_df = price_df.sort_values(by=["car_code", "Year"])

In [6]:
price_df["Next_Price_sd_scaled"] = price_df["Price_sd_scaled"].shift(-1)
price_df.drop(price_df.tail(1).index,inplace=True)

In [7]:

price_df

,car_code,car_manufacturer,car_model,car_model_year,Year,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,Next_Price_sd_scaled
372654,1,Lamborghini,Murcielago 6.2 V12,2002,19,0.817728,1.016211,0.979539,2021,-2.023064,-0.371702
186328,2,Fiat,Uno Mille Fire 1.0,2002,10,0.904178,0.852240,0.985393,2012,-0.371702,0.372936
207031,2,Fiat,Uno Mille Fire 1.0,2002,11,0.852240,0.907692,0.989107,2013,0.372936,-0.425508
227734,2,Fiat,Uno Mille Fire 1.0,2002,12,0.907692,0.879982,0.986298,2014,-0.425508,0.182580
248437,2,Fiat,Uno Mille Fire 1.0,2002,13,0.879982,0.904896,0.987452,2015,0.182580,0.222046
...,...,...,...,...,...,...,...,...,...,...,...
61773,22454,Mercedes-Benz,GLC 250 Highway 2.0,2018,3,1.075859,0.960412,0.922014,2021,3.201839,-1.952319
82476,22454,Mercedes-Benz,GLC 250 Highway 2.0,2018,4,0.960412,0.952021,0.930608,2022,-1.952319,-0.182192
20369,22456,Ford,Ka Trail 1.5,2018,1,0.785524,0.943683,0.917069,2019,-0.182192,0.429168
41072,22456,Ford,Ka Trail 1.5,2018,2,0.943683,1.135969,0.916243,2020,0.429168,4.289213


## Model first try --> ignore this!

In [8]:
price_df.sort_values(by=["car_manufacturer", "car_model", "car_model_year" , "calendar_year" ])

,car_code,car_manufacturer,car_model,car_model_year,Year,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,Next_Price_sd_scaled
1129,1165,Agrale,Marruá 2.8 Turbodiesel 4x4,2005,1,0.827534,0.955923,0.917069,2006,0.126351,1.450379
21832,1165,Agrale,Marruá 2.8 Turbodiesel 4x4,2005,2,0.955923,0.941984,0.916243,2007,1.450379,1.192389
42535,1165,Agrale,Marruá 2.8 Turbodiesel 4x4,2005,3,0.941984,0.923376,0.922014,2008,1.192389,1.393752
63238,1165,Agrale,Marruá 2.8 Turbodiesel 4x4,2005,4,0.923376,0.835827,0.930608,2009,1.393752,0.196289
83941,1165,Agrale,Marruá 2.8 Turbodiesel 4x4,2005,5,0.835827,0.895752,0.941001,2010,0.196289,-0.169465
...,...,...,...,...,...,...,...,...,...,...,...
114886,11612,W Motors,Fenyr SuperSport 3.7,2015,6,0.954017,1.016211,0.955932,2021,1.283366,-1.549119
135589,11612,W Motors,Fenyr SuperSport 3.7,2015,7,1.016211,1.081330,0.960582,2022,-1.549119,0.390098
135556,11579,W Motors,Lykan HyperSport 3.7,2013,7,0.892993,0.954017,0.960582,2020,-0.419926,1.254181
156259,11579,W Motors,Lykan HyperSport 3.7,2013,8,0.954017,1.016211,0.970114,2021,1.254181,-1.613198


In [9]:
price_df[price_df["car_code"] == 10123][price_df["Year"] == 1 ]

/tmp/ipykernel_17342/580542901.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  price_df[price_df["car_code"] == 10123][price_df["Year"] == 1 ]


,car_code,car_manufacturer,car_model,car_model_year,Year,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,Next_Price_sd_scaled
9895,10123,Volvo,XC90 Inscription T8 2.0,2019,1,0.778703,0.887521,0.917069,2020,-0.507204,-0.401075


In [10]:
#X = price_df[['car_manufacturer', 'car_model', 'Year', 'Price_YoY' ]]
#y = price_df[["Next_YoY_Price"]]

X = price_df[['car_manufacturer', 'car_model', 'Year', 'Price_sd_scaled' ]]
y = price_df[["Next_Price_sd_scaled"]]



In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder

In [14]:
def take_first_word(word):
    return word.split(" ")[0]
    

In [15]:
def take_first_word(word):
    return word.split(" ")[0]



# Impute then scale numerical values:
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean"))
    #,('standard_scaler', StandardScaler())
])

# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)




# Parallelize "num_transformer" and "cat_transfomer"
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, ['car_manufacturer', "car_model_small"]), #, 'car_model'  --> was removed
    #('num_transformer', num_transformer, [ 'Year','Price_YoY']) 
    ('num_transformer', num_transformer, [ 'Year','Price_sd_scaled']) 
])



In [16]:
from sklearn import set_config; set_config(display='diagram')
preprocessor

ColumnTransformer(transformers=[('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['car_manufacturer', 'car_model_small']),
                                ('num_transformer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Year', 'Price_sd_scaled'])])

In [17]:
X_train["car_model_small"] = X_train["car_model"].map(take_first_word)

X_train_transformed = preprocessor.fit_transform(X_train)

In [18]:
X_train_transformed.shape

(61523, 795)

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
model = LinearRegression()
model.fit(X_train_transformed, y_train)

LinearRegression()

In [21]:
model.coef_ 

array([[-4.38695329e+11, -1.77154064e+11, -1.02625385e+11,
        -8.38634006e+10, -1.66770734e+11, -2.15197348e+11,
        -1.76482246e+11, -1.52870085e+11, -1.43384381e+11,
        -1.56472957e+11, -9.87969099e+10, -1.79028491e+11,
        -1.32994517e+11, -1.14646905e+11, -1.46931317e+11,
        -2.51353618e+11, -2.51353618e+11, -1.23289597e+11,
        -1.63645786e+11, -1.64456613e+11, -1.55140011e+11,
        -1.21979046e+11, -2.20545355e+11, -1.79028491e+11,
         7.87232287e+10, -1.63645786e+11, -1.29194734e+11,
        -8.38634006e+10, -1.02074146e+11,  9.44682251e+11,
        -1.41372877e+11,  7.87232287e+10, -1.29714240e+11,
        -8.42284042e+10, -1.92917115e+11, -9.95186008e+10,
        -1.81608830e+11, -2.51353618e+11, -8.38634006e+10,
        -9.59776514e+10, -1.98103720e+11, -1.73567625e+11,
        -9.53234537e+10, -1.68220280e+11, -8.87220856e+11,
        -1.74141719e+11, -1.55127277e+11,  7.87232287e+10,
        -1.61395419e+11, -1.79028491e+11, -4.57787807e+1

In [22]:
model.intercept_

array([1.95397665e+11])

In [23]:
model.score(X_train_transformed,y_train)  

0.15641271725360884

In [24]:
car = price_df[price_df["car_model"] == "Golf Variant Highline 1.4 TSi AT"][price_df["Year"] == 4 ][price_df["car_code"] == 2924  ]
car["car_model_small"] = car["car_model"].map(take_first_word)
car

/tmp/ipykernel_17342/3470246675.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  car = price_df[price_df["car_model"] == "Golf Variant Highline 1.4 TSi AT"][price_df["Year"] == 4 ][price_df["car_code"] == 2924  ]


,car_code,car_manufacturer,car_model,car_model_year,Year,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,Next_Price_sd_scaled,car_model_small
64941,2924,Volkswagen,Golf Variant Highline 1.4 TSi AT,2015,4,0.834662,0.850733,0.930608,2019,-1.503667,-1.465515,Golf


In [25]:
car_trans = preprocessor.transform(car)

In [26]:
model.predict(car_trans)

array([[0.38421631]])

In [27]:
X_test["car_model_small"] = X_test["car_model"].map(take_first_word)
X_test

,car_manufacturer,car_model,Year,Price_sd_scaled,car_model_small
3823,Mitsubishi,Pajero Dakar 3.2 Turbo AT,1,-0.481528,Pajero
126346,Chevrolet,Captiva Sport 3.6 V6 4x2,7,-1.337603,Captiva
279816,Chevrolet,Omega CD 3.6 V6,14,0.549316,Omega
112056,Volkswagen,Space Cross 1.6 16V I-Motion,6,1.580467,Space
386142,Chevrolet,Astra Sedan CD 2.0 8V AT,19,0.278613,Astra
...,...,...,...,...,...
111975,Renault,Master Furgão Grand 2.3,6,1.015848,Master
113896,BMW,535i GT 3.0,6,0.252541,535i
28840,Jaguar,E-Pace 2.0,2,0.587664,E-Pace
125072,Fiat,Siena Fire 1.0,7,-0.075953,Siena


In [28]:
X_test_preprocess = preprocessor.transform(X_test)
X_test_preprocess

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        , -0.48152767],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         7.        , -1.33760345],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        14.        ,  0.54931614],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         2.        ,  0.58766447],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         7.        , -0.07595347],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         4.        , -1.97090339]])

In [29]:
model.score(X_test_preprocess, y_test)

-2.5331977738419053e+19

## Model with all other features

In [97]:
price_df = pd.read_csv('/home/marcnaweb/code/marcnaweb/car_recommendation_engine/raw_data/car_prices_w_prices_scaled.csv', index_col=0)

In [98]:
features_df = pd.read_csv('/home/marcnaweb/code/marcnaweb/car_recommendation_engine/raw_data/scaled_cleaned.csv')

### Merging the data frames 

In [99]:
merged_df = price_df.merge(features_df, left_on="car_code", right_on="car_code", how="left")

In [100]:
merged_df.rename(columns=lambda x: x.strip(), inplace=True)

In [101]:
merged_df[merged_df.index == 87887 ]

,car_code,car_manufacturer,car_model,car_model_year,Year_x,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,...,Maximum power,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year_y,Radio,Folding rear seat,Perimeter anti theft alarm
87887,441,Audi,A8 4.2 V8 Quattro,2003,1,0.749652,0.961737,0.917069,2004,-0.836267,...,0.167403,0.523084,0.033011,0.0,0.545455,0.062035,0.625,0.0,0.0,0.0


In [102]:
#merged_df["Next_Price_sd_scaled"] = merged_df["Price_sd_scaled"].shift(-1)
#merged_df.drop(merged_df.tail(1).index,inplace=True)

In [103]:
#price_df.columns

#### Making X and y

In [104]:
X = merged_df.drop( columns=['car_code' , 'car_model_year',  'Next_YoY_Price', 'Next_YoY_Pr_Pred',
       'Price_sd_scaled' ])


In [105]:
def take_first_word(word):
    return word.split(" ")[0]

X["car_model_small"] = X["car_model"].map(take_first_word)
X.drop(columns="car_model", inplace=True)

In [106]:
y = merged_df['Next_YoY_Price']
#y = merged_df["Next_Price_sd_scaled"]

In [107]:
X.columns
pd.options.display.max_rows = 500
columns = pd.DataFrame(  X.columns)


In [109]:
#num_feat = [feature for feature in X_train.select_dtypes(include='number').columns.tolist() if feature not in feat_eng + cyclical_feat + basement_features]

#categorical_features = list(X_train.select_dtypes(include='object').columns)


num_feat = [feature for feature in X.select_dtypes(include='number').columns.tolist() if feature not in ["Year_x", "calendar_year" ] ]
#num_feat
categorical_features = list(X.select_dtypes(include='object').columns)
categorical_features = [] # intentionaly removing these features ['car_manufacturer', 'car_model_small']
year_features = ["Year_x", "calendar_year" ]
year_features
categorical_features

[]

In [110]:

# Impute then scale numerical values:
num_inputer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean"))
    #('standard_scaler', StandardScaler())
])

standard_scaler = Pipeline([
    #('imputer', SimpleImputer(strategy="mean"))
    ('standard_scaler', StandardScaler())
])




# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)




# Parallelize "num_transformer" and "cat_transfomer"
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, categorical_features ), #, 'car_model'  --> was removed
    ('standard_scaler', standard_scaler, year_features ) ,
    ('num_inputer', num_inputer, num_feat  )  #numerical_columns 
])


#### training and testing the model

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:

X_train_transformed = preprocessor.fit_transform(X_train)

In [45]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_transformed, y_train)
model.score(X_train_transformed,y_train)  

0.3708195266701608

In [46]:
X_test_transformed = preprocessor.transform(X_test)


In [47]:
# we do not fit our test data!
model.score(X_test_transformed,y_test)  

-6.3981401265303085e+19

### trying with XG Boost

In [111]:
from xgboost import XGBRegressor

In [112]:
from sklearn.model_selection import train_test_split

X = preprocessor.fit_transform(X)  #keep in mind, is not procedural good

# Split data into train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42  # TEST = 30%
)

# Use the same function above for the validation set
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size = 0.5, random_state = 42  # TEST = 15%
)

In [113]:

#X_train_transformed = preprocessor.fit_transform(X_train)
#X_val_transformed = preprocessor.transform(X_val)
X.shape

(87891, 103)

In [114]:
X_train.shape
#X_train.shape

(61523, 103)

In [115]:
X_val.shape
#X_val.shape

(13184, 103)

In [129]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(max_depth=5, n_estimators=80, learning_rate=0.01)

xgb_reg.fit(X_train, y_train,
    # evaluate loss at each iteration
    eval_set=[(X_train, y_train), (X_val, y_val)],  
    # stop iterating when eval loss increases 5 times in a row
    early_stopping_rounds=3 
)

y_pred = xgb_reg.predict(X_val)

/home/marcnaweb/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:0.46033	validation_1-rmse:0.45936
[1]	validation_0-rmse:0.45580	validation_1-rmse:0.45485
[2]	validation_0-rmse:0.45131	validation_1-rmse:0.45038
[3]	validation_0-rmse:0.44687	validation_1-rmse:0.44596
[4]	validation_0-rmse:0.44247	validation_1-rmse:0.44158
[5]	validation_0-rmse:0.43812	validation_1-rmse:0.43725
[6]	validation_0-rmse:0.43381	validation_1-rmse:0.43296
[7]	validation_0-rmse:0.42955	validation_1-rmse:0.42871
[8]	validation_0-rmse:0.42533	validation_1-rmse:0.42451
[9]	validation_0-rmse:0.42115	validation_1-rmse:0.42035
[10]	validation_0-rmse:0.41701	validation_1-rmse:0.41623
[11]	validation_0-rmse:0.41292	validation_1-rmse:0.41215
[12]	validation_0-rmse:0.40886	validation_1-rmse:0.40811
[13]	validation_0-rmse:0.40485	validation_1-rmse:0.40412
[14]	validation_0-rmse:0.40088	validation_1-rmse:0.40017
[15]	validation_0-rmse:0.39695	validation_1-rmse:0.39625
[16]	validation_0-rmse:0.39306	validation_1-rmse:0.39238
[17]	validation_0-rmse:0.38921	validation

In [130]:
xgb_reg.score(X_test,  xgb_reg.predict(X_test) )

1.0

In [131]:
from sklearn.metrics import mean_squared_error

In [132]:
y_pred = xgb_reg.predict(X_test)

In [133]:
mean_squared_error(y_test, y_pred) ** (0.5)

0.21320794102373306

#### Let's see it in action

In [65]:
car = merged_df[merged_df["car_model_year"] == 2015 ] [merged_df["calendar_year"] == 2017]#[merged_df.index == 23575 ]
car = merged_df[merged_df.index == 60263 ]
car

/tmp/ipykernel_10273/3597691782.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  car = merged_df[merged_df["car_model_year"] == 2015 ] [merged_df["calendar_year"] == 2017]#[merged_df.index == 23575 ]


,car_code,car_manufacturer,car_model,car_model_year,Year_x,Price_YoY,Next_YoY_Price,Next_YoY_Pr_Pred,calendar_year,Price_sd_scaled,...,Maximum power,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year_y,Radio,Folding rear seat,Perimeter anti theft alarm
60263,2005,Peugeot,207 Passion In Concert 1.4,2015,2,0.844462,0.966102,0.916243,2017,-1.451522,...,0.027624,0.34035,0.027676,1.0,0.454545,0.111663,0.839286,1.0,1.0,0.0


In [66]:
car = car.drop( columns=['car_code' , 'car_model_year',  'Next_YoY_Price', 'Next_YoY_Pr_Pred',
       'Price_sd_scaled' ])

#car = car.drop( columns=['car_code' , 'car_model_year', 'Price_YoY' ,  'Next_YoY_Price', 'Next_YoY_Pr_Pred',
#       'Next_Price_sd_scaled' ])

car["car_model_small"] = car["car_model"].map(take_first_word)
car.drop(columns="car_model", inplace=True)
car

,car_manufacturer,Year_x,Price_YoY,calendar_year,Propulsion_Combustion,Propulsion_Electric,Propulsion_Hybrid,Propulsion_Light Hybrid,Propulsion_Plug-in hybrid,Car gearbox_Automated,...,Unit displacement,Trunk,USB connection,Gear speed transmissions,Urban,Year_y,Radio,Folding rear seat,Perimeter anti theft alarm,car_model_small
60263,Peugeot,2,0.844462,2017,1.0,0.0,0.0,0.0,0.0,0.0,...,0.34035,0.027676,1.0,0.454545,0.111663,0.839286,1.0,1.0,0.0,207


In [67]:
car_transformed = preprocessor.transform(car)

In [68]:
model.predict(car_transformed)

array([0.91959])